<a href="https://colab.research.google.com/github/sagihaider/Biogas_Prediction/blob/master/MLP_MISO_GridSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!git clone https://github.com/sagihaider/Biogas_Prediction.git 

import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dense
from sklearn.preprocessing import normalize
from keras import optimizers
from sklearn.model_selection import train_test_split

fatal: destination path 'Biogas_Prediction' already exists and is not an empty directory.


In [8]:
data = pd.read_excel('/content/Biogas_Prediction/Data/Gasification Data_Daya.xlsx', index_col=0, header=0)
data = data.iloc[1:223,:14] 
display(list(data.columns.values))

# Drop null balues and store dataframe in dataframe 2
data=data.dropna()
print(data)

#Check Null values again after removing
print(data.isnull().values.any())
print(data.isna().values.any())

X_all = data.iloc[:, :9]
y_all = data.iloc[:, 9:]

['C [%wt db]',
 'H [%wt db]',
 'O [%wt db]',
 'Moisture [%wt]',
 'Ash [%wt db]',
 'ER [-]',
 'T [ºC]',
 'Bed material',
 'Steam/Biomass',
 'H2 [%vol N2 free]',
 'CO [%vol N2 free]',
 'CO2 [%vol N2 free]',
 'CH4 [%vol N2 free]',
 'GY [Nm3/kg daf]']

        C [%wt db] H [%wt db]  ...  CH4 [%vol N2 free] GY [Nm3/kg daf]
Sr No.                         ...                                    
2            50.00        5.7  ...            6.474820        2.500000
3            50.00        5.7  ...            7.164179        2.500000
4            50.00        5.7  ...            6.474820        2.100000
5            50.00        5.7  ...            6.474820        2.400000
6            50.00        5.7  ...            8.181818        2.100000
...            ...        ...  ...                 ...             ...
219          46.85        6.3  ...            9.706147        0.947532
220          46.85        6.3  ...            9.455065        0.975752
221          46.76       5.68  ...            9.933682        0.971784
222          46.76       5.68  ...            8.778402        0.982989
223          46.76       5.68  ...            8.534014        0.973816

[187 rows x 14 columns]
False
False


In [10]:
input_columns = list(X_all.columns.values)
input_columns = [i.split(' [', 1)[0] for i in input_columns]
output_columns = list(y_all.columns.values)
output_columns = [i.split(' [', 1)[0] for i in output_columns]
print(input_columns, output_columns)

Xnorm = normalize(X_all, 'l2', axis=1)
ynorm = normalize(y_all, 'l2', axis=1)
Y = ynorm[:,1]

['C', 'H', 'O', 'Moisture', 'Ash', 'ER', 'T', 'Bed material', 'Steam/Biomass'] ['H2', 'CO', 'CO2', 'CH4', 'GY']


In [21]:
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.wrappers.scikit_learn import KerasRegressor

# Function to create model, required for KerasClassifier
def create_model(optimizer='adam', activation = 'sigmoid', h_neu = 2):
  # create model
  model = Sequential()
  model.add(Dense(10, input_shape = (9,), activation = activation))
  # model.add(Dense(10, activation = 'sigmoid'))
  model.add(Dense(h_neu, activation = activation))
  model.add(Dense(1))
  # model.summary()
  # sgd = optimizers.SGD(lr = 0.01)    # stochastic gradient descent optimizer
  model.compile(optimizer = optimizer, loss = 'mean_squared_error', metrics = ['mse']) 
  return model

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# create model
model = KerasRegressor(build_fn=create_model, epochs=100, batch_size=10, verbose=0)

optimizer = ['SGD', 'RMSprop', 'Adagrad']
activation = ['sigmoid', 'tanh', 'relu', 'softplus']
h_neu = [2,4,6,8,10]
param_grid = dict(optimizer=optimizer, activation=activation, h_neu=h_neu)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(Xnorm, Y)

#summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -0.012694 using {'activation': 'relu', 'h_neu': 10, 'optimizer': 'RMSprop'}
-0.018031 (0.011161) with: {'activation': 'sigmoid', 'h_neu': 2, 'optimizer': 'SGD'}
-0.017486 (0.011284) with: {'activation': 'sigmoid', 'h_neu': 2, 'optimizer': 'RMSprop'}
-0.053400 (0.064075) with: {'activation': 'sigmoid', 'h_neu': 2, 'optimizer': 'Adagrad'}
-0.018132 (0.011286) with: {'activation': 'sigmoid', 'h_neu': 4, 'optimizer': 'SGD'}
-0.017120 (0.010629) with: {'activation': 'sigmoid', 'h_neu': 4, 'optimizer': 'RMSprop'}
-0.327879 (0.472371) with: {'activation': 'sigmoid', 'h_neu': 4, 'optimizer': 'Adagrad'}
-0.018246 (0.011475) with: {'activation': 'sigmoid', 'h_neu': 6, 'optimizer': 'SGD'}
-0.017172 (0.010741) with: {'activation': 'sigmoid', 'h_neu': 6, 'optimizer': 'RMSprop'}
-0.138203 (0.157493) with: {'activation': 'sigmoid', 'h_neu': 6, 'optimizer': 'Adagrad'}
-0.017949 (0.011098) with: {'activation': 'sigmoid', 'h_neu': 8, 'optimizer': 'SGD'}
-0.017630 (0.012105) with: {'activation': 's